In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb
gbm = lgb.LGBMRanker()
from sklearn.model_selection import train_test_split

In [3]:
# Specify the file name
file_name = 'df_OptionB.csv'

# Specify the full file path
file_path = r'/Users/frederike/Documents/Artificial inteligence (Master)/Data mining techniques/Data-mining-assignment-2/' + file_name

# Read the dataset using pandas
df = pd.read_csv(file_path)

In [4]:
df.columns

Index(['srch_id', 'prop_id', 'prop_starrating', 'prop_review_score',
       'prop_brand_bool', 'prop_location_score1', 'prop_location_score2',
       'prop_log_historical_price', 'price_usd', 'promotion_flag',
       'srch_length_of_stay', 'srch_booking_window', 'srch_adults_count',
       'srch_room_count', 'srch_saturday_night_bool',
       'orig_destination_distance', 'click_bool', 'booking_bool', 'score',
       'avg_comp_rate', 'avg_comp_inv', 'abroad_bool', 'children_bool',
       'srch_query_affinity_score_low', 'srch_query_affinity_score_high',
       'starrating_diff_low', 'starrating_diff_high', 'usd_diff_low',
       'usd_diff_high'],
      dtype='object')

#### This part of the code makes sure it is a balanced data set, that has a comparable amount of click and non click 

In [5]:
click_indices = df[df['click_bool'] == 1].index
random_indices = np.random.choice(click_indices, len(df.loc[df['click_bool'] == 1]), replace=False)
click_sample = df.loc[random_indices]

not_click = df[df['click_bool'] == 0].index
random_indices = np.random.choice(not_click, sum(df['click_bool']), replace=False)
not_click_sample = df.loc[random_indices]

df = pd.concat([not_click_sample, click_sample], axis=0)

print("Non-click impressions: ", len(df[df['click_bool'] == 0]))
print("Click impression: ", len(df[df['click_bool'] == 1]))
print("Total number of records in resampled data: ", len(df))

Non-click impressions:  221879
Click impression:  221879
Total number of records in resampled data:  443758


In [6]:
df = df.sort_values(by='srch_id')

In [7]:
group = list(df['srch_id'].value_counts())

In [8]:
y=df['score']
X=df.drop(['score'], axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=1)

In [9]:
group_train = list(X_train['srch_id'].value_counts())
group_test = list(X_test['srch_id'].value_counts())

In [10]:
X_train = X_train.drop(['click_bool', 'booking_bool', 'srch_id', 'prop_id'], axis=1)
X_test = X_test.drop(['click_bool', 'booking_bool', 'srch_id', 'prop_id'], axis=1)

In [11]:
print(len(X_train))
print(len(X_test))

284004
88752


In [12]:
gbm.fit(X_train, y_train, group=group_train,
        eval_set=[(X_train, y_train), (X_test, y_test)], eval_group=[group_train, group_test],
        eval_at=5, early_stopping_rounds=50)

/Users/frederike/Library/Python/3.9/lib/python/site-packages/lightgbm/sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "


[1]	training's ndcg@5: 0.947995	valid_1's ndcg@5: 0.984438
[2]	training's ndcg@5: 0.949004	valid_1's ndcg@5: 0.984655
[3]	training's ndcg@5: 0.949498	valid_1's ndcg@5: 0.984728
[4]	training's ndcg@5: 0.949806	valid_1's ndcg@5: 0.984847
[5]	training's ndcg@5: 0.949987	valid_1's ndcg@5: 0.984883
[6]	training's ndcg@5: 0.950255	valid_1's ndcg@5: 0.984993
[7]	training's ndcg@5: 0.950491	valid_1's ndcg@5: 0.984971
[8]	training's ndcg@5: 0.950694	valid_1's ndcg@5: 0.984976
[9]	training's ndcg@5: 0.950933	valid_1's ndcg@5: 0.985127
[10]	training's ndcg@5: 0.951158	valid_1's ndcg@5: 0.985057
[11]	training's ndcg@5: 0.951271	valid_1's ndcg@5: 0.985138
[12]	training's ndcg@5: 0.951458	valid_1's ndcg@5: 0.985172
[13]	training's ndcg@5: 0.951581	valid_1's ndcg@5: 0.985119
[14]	training's ndcg@5: 0.951616	valid_1's ndcg@5: 0.98518
[15]	training's ndcg@5: 0.951751	valid_1's ndcg@5: 0.985184
[16]	training's ndcg@5: 0.951974	valid_1's ndcg@5: 0.985127
[17]	training's ndcg@5: 0.952091	valid_1's ndcg@5:

LGBMRanker()

In [13]:
test_pred = gbm.predict(X_test)

In [14]:
X_test["predicted_ranking"] = test_pred
X_test.sort_values("predicted_ranking", ascending=False)

,prop_starrating,prop_review_score,prop_brand_bool,prop_location_score1,prop_location_score2,prop_log_historical_price,price_usd,promotion_flag,srch_length_of_stay,srch_booking_window,...,avg_comp_inv,abroad_bool,children_bool,srch_query_affinity_score_low,srch_query_affinity_score_high,starrating_diff_low,starrating_diff_high,usd_diff_low,usd_diff_high,predicted_ranking
2234173,3,4.5,1,1.10,0.3701,4.77,103.00,0,1,0,...,0.5,0,1,0,0,1,0,1,0,2.447179
2530284,4,4.0,1,0.00,0.9286,4.85,103.87,0,1,3,...,0.0,0,0,0,0,0,1,1,0,2.190147
2112987,3,4.5,1,1.79,0.8982,4.91,125.00,0,1,0,...,0.0,0,0,0,0,0,1,1,0,2.187304
687157,3,4.0,1,0.00,0.9549,4.39,70.00,1,1,24,...,0.0,0,1,0,0,1,0,1,0,2.185414
1919318,2,3.0,1,1.79,0.3851,4.10,65.00,0,1,0,...,0.0,0,1,0,0,1,0,1,0,2.180960
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2659474,0,0.0,0,2.89,0.0000,4.49,348.92,0,7,3,...,0.0,1,0,0,0,0,0,0,0,-1.556606
3249937,0,0.0,0,4.09,0.0000,5.40,76.24,0,2,5,...,0.0,0,0,0,0,0,1,0,1,-1.571111
411026,3,4.0,0,1.39,0.0007,4.57,138.50,0,3,60,...,0.0,0,1,0,0,0,0,0,0,-1.598757
3427462,3,4.5,1,1.95,0.0000,4.91,595.00,0,1,3,...,0.5,0,0,0,0,0,0,0,0,-1.629694


In [15]:
file_name = 'df_testB.csv'
file_path = r'/Users/frederike/Documents/Artificial inteligence (Master)/Data mining techniques/Data-mining-assignment-2/' + file_name
df_test = pd.read_csv(file_path)

In [16]:
# Sort test set 
df_test = df_test.sort_values(by='srch_id')

# Creating group list to use in test set prediction
group = list(df_test['srch_id'].value_counts())

# Storing the removed columns in a separate DataFrame
df_test_columns_removed = df_test[['srch_id', 'prop_id']].copy()

# Create a new DataFrame by dropping the desired columns
df_test = df_test.drop(columns=['srch_id', 'prop_id'])

In [17]:
# Apply model to test set
pred = gbm.predict(df_test, group=group)

In [18]:
# Add predictions to test set
df_test["predicted_ranking"] = pred

# Adding the columns back
df_test['srch_id'] = df_test_columns_removed['srch_id']
df_test['prop_id'] = df_test_columns_removed['prop_id']

# Sort the dataframe by `srch_id` and `predicted_ranking` in descending order
test_sorted = df_test.sort_values(['srch_id', 'predicted_ranking'], ascending=[True, False])

In [19]:
# Select the columns `srch_id` and `prop_id`
result = test_sorted[['srch_id', 'prop_id']]
print(len(result))

4959183


In [20]:
result.to_csv('LambdaMart_result_balanced.csv', index=False)

price_usd 466
prop_location_score2 413
prop_location_score1 388
prop_log_historical_price 277
prop_starrating 246
prop_review_score 185
orig_destination_distance 164
srch_booking_window 135
promotion_flag 113
srch_adults_count 105
avg_comp_rate 96
srch_length_of_stay 70
srch_room_count 66
prop_brand_bool 63
avg_comp_inv 46
children_bool 43
starrating_diff_low 26
abroad_bool 26
usd_diff_high 21
srch_query_affinity_score_high 16
usd_diff_low 11
srch_query_affinity_score_low 11
srch_saturday_night_bool 8
starrating_diff_high 5
